# Table 2

This script calculates the local break-even time based on the radiative forcing of albedo changes, carbon emission suppression from replacing emissions in the fossil fuel based energy sector with PV electricity, and afforestation NEP carbon sink.

In [1]:
import pandas as pd
import numpy as np
import glob
import scipy.stats
import gmpy2
from gmpy2 import mpfr

In [8]:
# Constants

# Ketura PV field electricity production
daily_average = mpfr (25423.11, 120)               # kWh day -1, total daily average electricity production, data from company
daily_rounded = mpfr (25500.00, 120)               # kWh day-1, rounding the above value
area_pv_ketura = mpfr (75600, 120)                 # m2, area of the PV field in Ketura
PVe_m2 = daily_rounded/area_pv_ketura              # kWh m-2 day-1, the daily electricity output of the Ketura field divided by the total area
EP_a = PVe_m2 * 365                                # kWh m-2 year-1, Electricity Production annual (daily_rounded/area_pv_ketura) * 365, gives the annual electricity production of the field per m2, equation 2.8 of my thesis

# Life-cycle assessment of PV
LCA = 0.039                                        # kgCO2eq kWh-1, amount of CO2eq emitted by the production, transport, operation, disposal and recycling of the PV panels (Meier and Kulcinski, 2002)
LCA_storage = 0.095                                # kgCO2eq kWh-1, amount of CO2eq emitted PV electricity production considering all the steps above + all the steps of storage (Lima, 2021)

# Storage Performance Ratio
PR_storage = 0.8                                   # 0.6-0.95, depending on the storage techbology (Lima, 2021)

# Amount of CO2 emitted by each kWh generated by different sources (International Energy Agency, 2019)
FCO2_coal = 0.87                                  # Coal [kgCO2 kWh-1] (International Energy Agency, 2019, Page I.21)
FCO2_gas = 0.40                                   # Gas [kgCO2 kWh-1] (International Energy Agency, 2019, Page I.21)
FCO2_world = 0.54                                 # World [kgCO2 kWh-1] (International Energy Agency, 2019, Page II.67)
FCO2_oil = 0.67

# Afforestation
C_afforestation = 0.15                            # Carbon sequestration [kg carbon m-2 year-1] (Qubaja, 2020)

Eg = 245                                           # Insolation [W m^-2], IMS
albedo_yatir_forest = 0.12                         # unitless
albedo_yatir_desert = 0.28                         # unitless
albedo_ketura_pv = 0.17                            # unitless
albedo_ketura_desert = 0.38                        # unitless
delta_alb_yatir = albedo_yatir_forest - albedo_yatir_desert
delta_alb_ketura = albedo_ketura_pv - albedo_ketura_desert
PVeff = 0.058                                      # unitless, is the ratio between PVe_annual and the mean annual solar irradiation of the region (2122 kWh m-2, Vardimon 2011)

#g = 0.0010368                                     # conversion from [umol m^-2 s^-1] to [kgC m^-2 d^-1]
#fgc = 0.0216                                      # conversion from [umol m^-2 s^-1] to [gC m^-2 30min^-1]

### Functions

In [9]:
# (1) Radiative forcing
# - - - - - - - - - - -

def list_to_float(lst):
    out_lst = [float(i) for i in lst]
    return(out_lst)

# Carbon Emission Suppression: The amount of C which would be emitted annualy to the atmosphere
# if the same amount of energy would be produced by an FCO2 source (coal, gase, world mixture). Requires:
# - Annual electricity Production (kWh m-2 year-1)
# - Amount of CO2 emitted by the production, transport, operation, disposal and recycling of the PV panels (Meier and Kulcinski, 2022) 
def calculate_CES(EP_a, FCO2, LCA):
    CO2_to_C = 0.272727273 # Conversion factor, 12/44 g mol-1 (molar mass of C/CO2)
    # Calculate CES
    CES = EP_a * (FCO2 - LCA) * CO2_to_C # kgC m-2 year-1
    return(CES)

# Equation 1
# I have to apply the transmitance factor to the product (delta albedo * Eg), because the Sn is
# Sn = SWin_desert - SWin_pvfield - SWout_desert - SWout_pvfield, and SWin is equal in both
# desert and pv field. Therefore, Sn is only SWout_desert - SWout_pvfield, and transmitance factor
# applies to SWout only (because SWin already passed thgough the atmosphere, and regarding 
# global warming, we need to do the calculations at the toa). (delta albedo * Eg) is equivalent
# to SWout, and that`s why I need to apply the transmittance factor.

def calculate_breakeven_time_etminan(delta_albedo, PVeff, insolation, CES, RF_t=0):
    # Set calculation precision
    gmpy2.get_context().precision=120
    # Constants
    Area_earth = mpfr('5.1e14',120)     # Surface area of planet Earth [m^2]
    k = mpfr('2.16e12',120)             # [kgC ppm^-1] (Nemet)
    af = 0.46                           # airborne fraction (Friedlingstein 2020, Global Carbon Project)
    C0_ppm = mpfr('409.85',120)         # base background CO2 [ppm] in 2019 https://www.esrl.noaa.gov/gmd/ccgg/trends/gl_data.html
    N2O = 331                           # base background N2O [ppb] in 2019 https://www.esrl.noaa.gov/gmd/ccgg/trends_n2o/
    
    # Radiative forcing of the albedo change
    RFr_global = -(delta_albedo * insolation + 0.1*RF_t)/Area_earth
    RFr_local  = -(delta_albedo - PVeff) * insolation + 0.1*RF_t
    
    # Radiative forcing of carbon emission suppression OR sequestration
    delta_C  = CES*af/k
    
    # Change in atmospheric carbon due to the change in surface cover
    C_atm = delta_C + C0_ppm
    
    # Etminan carbon radiative forcing equation
    a = mpfr('-2.4e-7', 120)   # [W m^-2 ppm^-1]
    b = mpfr('7.2e-4', 120)    # [W m^-2 ppm^-1]
    d = mpfr('-2.1e-4', 120)   # [W m^-2 ppb^-1]

    nu_etminan = a*(C_atm - C0_ppm)**2 + b*np.abs(C_atm - C0_ppm) + d*N2O + 5.36 # (Etminan et al., 2016)
    RF_C = nu_etminan * gmpy2.log(C_atm/C0_ppm)
    
    # Break-even time: Years to balance calculation
    y = RFr_global / RF_C
    
    return(list_to_float([RFr_global,PVeff, RFr_local, delta_C, C_atm, nu_etminan, RF_C, y]))

# Allows to run multiple rows of data from a df, and outputs a format that can be put in a df
def calculate_breakeven_time_df(delta_albedo, PVeff, insolation, CES, RF_t=0, method='etminan'):
    out_lst = []
    for index, value in delta_albedo.items():
        if(method == 'etminan'):
            out_lst.append(calculate_breakeven_time_etminan(delta_albedo.iloc[index], \
                                                            PVeff.iloc[index], \
                                                            insolation.iloc[index], \
                                                            CES.iloc[index], \
                                                            RF_t.iloc[index]))
        elif(method == 'bright'):
            out_lst.append(calculate_breakeven_time_bright( delta_albedo.iloc[index], \
                                                            PVeff.iloc[index], \
                                                            insolation.iloc[index], \
                                                            CES.iloc[index], \
                                                            RF_t.iloc[index]))
        else:
            raise NameError('ERROR: Unknown method specified. Try etminan or bright. Standard is etminan')
    # Transpose the list of lists
    out_lst_t = list(map(list, zip(*out_lst)))
    return(out_lst_t)

### Run the code

In [10]:
# Create df and fill it
df = pd.DataFrame(data={'Location': ['Yatir', 'Ketura', 'Ketura', 'Ketura','Ketura'],\
                        'Scenario': ['Afforestation', 'Coal', 'Gas', 'World','Oil']})

# Add delta albedo
df['delta_albedo'] = np.nan
df.loc[df['Location'] == 'Yatir', 'delta_albedo']  = delta_alb_yatir
df.loc[df['Location'] == 'Ketura', 'delta_albedo'] = delta_alb_ketura

# Add PVeff
df['PVeff'] = np.nan
df.loc[df['Location'] == 'Yatir', 'PVeff']  = 0
df.loc[df['Location'] == 'Ketura', 'PVeff'] = PVeff

# Add insolation
df['insolation'] = Eg

# Amount of CO2 emitted by each kWh generated by different energy sources
df['FCO2'] = np.nan
df.loc[df['Scenario'] == 'Afforestation', 'FCO2']  = np.nan # No CO2 emitted by afforestation
df.loc[df['Scenario'] == 'Coal', 'FCO2']  = FCO2_coal
df.loc[df['Scenario'] == 'Gas', 'FCO2']   = FCO2_gas
df.loc[df['Scenario'] == 'World', 'FCO2'] = FCO2_world
df.loc[df['Scenario'] == 'Oil', 'FCO2'] = FCO2_oil

# Add carbon emission suppression or sequestration
df['C_emiss_seq'] = calculate_CES(EP_a, df['FCO2'], LCA_storage)
df.loc[df['Scenario'] == 'Afforestation', 'C_emiss_seq']  = C_afforestation  # No CES calculation for afforestation, replace value

# RF_t = Radiative Forcing caused by the thermal radiation effect (in Yatir, its emission is supressed by a lower Ts, increasing the Rn)
df['RF_t'] = np.nan
df.loc[df['Location'] == 'Yatir', 'RF_t']  = 1.8
df.loc[df['Location'] == 'Ketura', 'RF_t'] = 0

# Now fill with calculations
df['RFr_global'], df['PVeff'], df['RFr_local'], df['delta_C'], df['C_atm'], df['nu_etminan'], df['RF_C'], df['y'] = \
    calculate_breakeven_time_df(df['delta_albedo'], df['PVeff'], df['insolation'], df['C_emiss_seq'], df['RF_t'])

display(df)

,Location,Scenario,delta_albedo,PVeff,insolation,FCO2,C_emiss_seq,RF_t,RFr_global,RFr_local,delta_C,C_atm,nu_etminan,RF_C,y
0,Yatir,Afforestation,-0.16,0.000,245,NaN,0.15,1.8,7.650980e-14,39.38,3.194444e-14,409.85,5.29049,4.123503e-16,185.545656
1,Ketura,Coal,-0.21,0.058,245,0.87,26.022050891822916676268421686046289138,0.0,1.008824e-13,65.66,5.541733e-12,409.85,5.29049,7.153467e-14,1.410258
2,Ketura,Gas,-0.21,0.058,245,0.40,10.240936157427084683922230213497788149,0.0,1.008824e-13,65.66,2.180940e-12,409.85,5.29049,2.815235e-14,3.583443
3,Ketura,World,-0.21,0.058,245,0.54,14.941693737885418331642428449313907632,0.0,1.008824e-13,65.66,3.182027e-12,409.85,5.29049,4.107474e-14,2.456068
4,Ketura,Oil,-0.21,0.058,245,0.67,19.306682919739585023969893188383807462,0.0,1.008824e-13,65.66,4.111608e-12,409.85,5.29049,5.307411e-14,1.900783
